# MDP For Maze Solving

Given: Randomly generated Maze defined by an nxn matrix with either 1 or inf. Indicating whether you can or cannot travel to that position.

Find a shortest path from origin (0,0) to end (n-1,n-1)

## Submission Guide:
* Do not modify function signature.
* Make sure you are not using global variables.

# Maze Generation (Do Not Modify)

In [1]:
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
np.random.seed(42)

def generate_maze(maze_size=8):
    maze = np.random.choice([1, np.inf], size=(maze_size, maze_size), p=[0.6, 0.4])

    # Ensure a path from (0,0) to (255,255)
    x, y = 0, 0
    maze[x, y] = 1
    while x < maze_size - 1 or y < maze_size - 1:
        if x < maze_size - 1 and (y == maze_size - 1 or np.random.rand() < 0.5):
            x += 1
        else:
            y += 1
        maze[x, y] = 1
    maze[maze_size - 1, maze_size - 1] = 1
    return maze

In [2]:
def get_char(maze, x, y):
    maze_size = maze.shape[0]
    if x == y == 0:
        return 'S'  # Start
    elif x == maze_size - 1 and y == maze_size - 1:
        return 'E'  # End
    elif maze[x, y] == 1:
        return '.'  # Open path
    elif maze[x,y] == 0.5:
        return '*'  # Path taken
    else:
        return '#'
        
def print_maze(maze,highlight=-1):
    maze_size = maze.shape[0]
    print("↓")
    for i in range(maze_size):
        for j in range(maze_size):
            if highlight!=-1 and highlight==(i*maze_size+j):
                    print("X", end=' ')
            else:
                print(get_char(maze, i, j), end=' ')
            if i == maze_size - 1 and j == maze_size - 1:
                print("→", end=' ')
        print()
    print()

def print_np_array(arr):
    for row in arr:
        print(' '.join(f"{val:6.1f}" for val in row))
    print()


In [3]:
maze = generate_maze(maze_size=16)
print_maze(maze)

↓
S . . . . . . # # # . # # . . . 
. . . . . . . . . # . . . . # . 
. # # # . . . . . . . # . # . . 
. . # # # # . # . . . . . . # . 
. . . # . # # . . # . . . . . . 
# # . . . . # # # . . # # . # . 
. . . . . # . . # . . # . . . . 
# # # # # . # . # # . . . . # . 
. . . . . . # . . # . # # . . . 
. . # . . . # . . . # . # # . . 
. # # . . # . . . . # . . . # . 
# . # . . . # # . # # . . . . . 
# # . . # # # # # . . # # . . . 
. . . # # . # . . # # # # . . . 
. . # . # # # . . . . # . . # . 
# # # . . # . # # # . . # . . E → 



In [4]:
# Convert maze to state transition matrix
def generate_PR(maze):
    maze_size = maze.shape[0]
    num_states = maze_size * maze_size
    P = np.zeros((num_states, num_states))
    for i in range(maze_size):
        for j in range(maze_size):
            if maze[i, j] == 1:
                state = i * maze_size + j
                for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    ni, nj = i + di, j + dj
                    if 0 <= ni < maze_size and 0 <= nj < maze_size and maze[ni, nj] == 1:
                        next_state = ni * maze_size + nj
                        P[state, next_state] = 1
                if P[state].sum() > 0:
                    P[state] /= P[state].sum()  # Normalize to get probabilities
    # Define rewards    
    P_check = P.sum(axis=1)
    print("Row sums (should be 1.0 for valid states):", P_check.min(), P_check.max())
    R = np.full(num_states, -1.0)
    goal_state = num_states - 1
    R[goal_state] = 100.0  
    return P, R
P, R = generate_PR(maze)
print("Transition matrix P shape:", P.shape)
print("Reward vector R:", R)

Row sums (should be 1.0 for valid states): 0.0 1.0
Transition matrix P shape: (256, 256)
Reward vector R: [ -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.
  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  

In [5]:
highlight = 17
print_maze(maze,highlight=highlight)
print_np_array(P[highlight].reshape(maze.shape))

↓
S . . . . . . # # # . # # . . . 
. X . . . . . . . # . . . . # . 
. # # # . . . . . . . # . # . . 
. . # # # # . # . . . . . . # . 
. . . # . # # . . # . . . . . . 
# # . . . . # # # . . # # . # . 
. . . . . # . . # . . # . . . . 
# # # # # . # . # # . . . . # . 
. . . . . . # . . # . # # . . . 
. . # . . . # . . . # . # # . . 
. # # . . # . . . . # . . . # . 
# . # . . . # # . # # . . . . . 
# # . . # # # # # . . # # . . . 
. . . # # . # . . # # # # . . . 
. . # . # # # . . . . # . . # . 
# # # . . # . # # # . . # . . E → 

   0.0    0.3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.3    0.0    0.3    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
   0.0    0.0    0.

## Value Iteration and Extract Policy

In [9]:
def value_iteration(P, R, gamma=0.9, theta=1e-6):
    num_states = P.shape[0]
    V = np.zeros(num_states)
    while True:
        delta = 0
        V_new = R + gamma * P.dot(V)   # Bellman update
        delta = np.max(np.abs(V_new - V))
        V = V_new
        if delta < theta:
            break
    return V
V = value_iteration(P, R)


In [10]:

maze_size = maze.shape[0]
V = V.reshape((maze_size, maze_size))
print_np_array(V)
V = V.flatten()


 -10.0  -10.0  -10.0  -10.0  -10.0  -10.0  -10.0   -1.0   -1.0   -1.0  -10.0   -1.0   -1.0  -10.0  -10.0  -10.0
 -10.0  -10.0  -10.0  -10.0  -10.0  -10.0  -10.0  -10.0  -10.0   -1.0  -10.0  -10.0  -10.0  -10.0   -1.0  -10.0
 -10.0   -1.0   -1.0   -1.0  -10.0  -10.0  -10.0  -10.0  -10.0  -10.0  -10.0   -1.0  -10.0   -1.0  -10.0  -10.0
 -10.0  -10.0   -1.0   -1.0   -1.0   -1.0  -10.0   -1.0  -10.0  -10.0  -10.0  -10.0  -10.0  -10.0   -1.0   -9.9
 -10.0  -10.0  -10.0   -1.0  -10.0   -1.0   -1.0  -10.0  -10.0   -1.0  -10.0  -10.0  -10.0   -9.9   -9.9   -9.9
  -1.0   -1.0  -10.0  -10.0  -10.0  -10.0   -1.0   -1.0   -1.0  -10.0  -10.0   -1.0   -1.0   -9.9   -1.0   -9.7
 -10.0  -10.0  -10.0  -10.0  -10.0   -1.0  -10.0  -10.0   -1.0  -10.0  -10.0   -1.0   -9.8   -9.8   -9.7   -9.5
  -1.0   -1.0   -1.0   -1.0   -1.0  -10.0   -1.0  -10.0   -1.0   -1.0   -9.9   -9.9   -9.8   -9.6   -1.0   -8.9
 -10.0  -10.0  -10.0  -10.0  -10.0  -10.0   -1.0  -10.0  -10.0   -1.0   -9.9   -1.0   -1.0   -9.2   -8.5

In [11]:
def policy_extraction(P, V, gamma=0.9):
    num_states = P.shape[0]
    policy = np.zeros(num_states, dtype=int)
    for s in range(num_states):
        # Possible next states = indices with non-zero transition prob from s
        next_states = np.flatnonzero(P[s])

        if next_states.size == 0:
            # No outgoing edges: stay put
            policy[s] = s
        else:
            # Greedy one-step choice: pick neighbor with highest V
            best_next = next_states[np.argmax(V[next_states])]
            policy[s] = best_next
    return policy

policy = policy_extraction(P, V)
print(policy)


[  1   2   3   4   5   6  22   7   8   9  26  11  12  14  15  31  17  18
  19  20  21  22  23  24  40  25  27  28  44  13  30  47  16  33  34  35
  37  38  39  40  41  42  58  43  60  45  47  63  32  48  50  51  52  53
  38  55  57  58  59  60  61  77  62  79  48  49  65  67  84  69  70  72
  56  73  90  76  77  93  79  95  80  81  66  82  83  84  86  87  88 105
 106  91  92 109  94 111  97  98  82  98  99 101 103 102 104 106 122 107
 109 125 111 127 112 113 114 115 116 133 118 135 120 121 123 124 125 141
 126 143 129 128 129 130 131 117 134 119 135 137 122 139 140 142 158 159
 128 129 146 131 132 133 150 135 136 152 154 171 156 157 159 175 144 161
 162 147 148 165 167 151 167 153 170 172 173 189 174 191 176 177 178 163
 164 180 182 183 168 185 186 188 189 205 206 207 192 193 195 179 196 197
 198 199 200 202 201 203 204 221 222 223 209 208 194 211 212 213 214 216
 215 217 218 219 220 237 223 239 208 209 226 243 228 229 230 215 216 232
 233 235 237 253 238 255 240 241 242 227 243 245 24

In [12]:
def map_state_to_coord(state, maze_size):
    return (state // maze_size, state % maze_size)

def extract_path(maze, policy):
    maze_size = maze.shape[0]
    start = 0
    goal = len(policy) - 1

    path = [map_state_to_coord(start, maze_size)]
    state = start
    visited = {start}
    safety_cap = len(policy) + 5  # prevent infinite loops

    while state != goal and safety_cap > 0:
        nxt = policy[state]
        # stop if stuck (self-loop) or cycling
        if nxt == state or nxt in visited:
            break
        state = nxt
        visited.add(state)
        path.append(map_state_to_coord(state, maze_size))
        safety_cap -= 1

    return path

path = extract_path(maze, policy)
output_maze = maze.copy()
for (x, y) in path:
    output_maze[x, y] = 0.5  # Mark the path
print_maze(maze)
print_maze(output_maze)

↓
S . . . . . . # # # . # # . . . 
. . . . . . . . . # . . . . # . 
. # # # . . . . . . . # . # . . 
. . # # # # . # . . . . . . # . 
. . . # . # # . . # . . . . . . 
# # . . . . # # # . . # # . # . 
. . . . . # . . # . . # . . . . 
# # # # # . # . # # . . . . # . 
. . . . . . # . . # . # # . . . 
. . # . . . # . . . # . # # . . 
. # # . . # . . . . # . . . # . 
# . # . . . # # . # # . . . . . 
# # . . # # # # # . . # # . . . 
. . . # # . # . . # # # # . . . 
. . # . # # # . . . . # . . # . 
# # # . . # . # # # . . # . . E → 

↓
S * * * * * * # # # . # # . . . 
. . . . . . * * * # . . . . # . 
. # # # . . . . * * * # . # . . 
. . # # # # . # . . * * * * # . 
. . . # . # # . . # . . . * . . 
# # . . . . # # # . . # # * # . 
. . . . . # . . # . . # . * . . 
# # # # # . # . # # . . . * # . 
. . . . . . # . . # . # # * * . 
. . # . . . # . . . # . # # * * 
. # # . . # . . . . # . . . # * 
# . # . . . # # . # # . . . . * 
# # . . # # # # # . . # # . . * 
. . . # # . # . . # # # # . . * 
. .